In [1]:
from utils.hints import hide_toggle

# Набор данных "Auction Verification"

Датасет был создан в рамках научного исследования с целью выяснить, можно ли заменить дорогостоящую верификацию сложных моделей процессов (в данном случае: одновременные многораундовые аукционы, используемые для аукционов частотных спектров) предсказаниями результатов.

Каждый экземпляр в этом датасете представляет один запуск верификации. Верификация выполняется для проверки, является ли определенная цена возможной для определенного продукта, и в некоторых случаях, определения, может ли определенный участник выиграть продукт по этой цене. Каждый экземпляр содержит детали одного такого сценария верификации.

Краткое описание признаков:

- process.b1.capacity: Характеристика (вместимость) первого участника (заявителя). Целое число. Вместимость (максимальное количество продуктов для выигрыша) участника 1.

- process.b2.capacity: Характеристика (вместимость) второго участника (заявителя). Целое число. Вместимость (максимальное количество продуктов для выигрыша) участника 2.

- process.b3.capacity: Характеристика (вместимость) третьего участника (заявителя). Целое число. Вместимость (максимальное количество продуктов для выигрыша) участника 3.

- process.b4.capacity: Характеристика (вместимость) четвертого участника (заявителя). Целое число. Вместимость (максимальное количество продуктов для выигрыша) участника 4.

- property.price: Характеристика (цена) продукта. Целое число. Текущая проверенная цена.

- property.product: Характеристика (продукт) продукта. Целое число. Текущий проверенный продукт.

- property.winner: Характеристика (победитель) продукта. Целое число. Участник (заявитель), который в настоящее время признан победителем продукта (0, если проверена только цена).

Целевая переменная:

- verification.result: Целевой атрибут (категориальный). Результат верификации. Бинарный результат - возможен ли проверенный результат?

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score

# Загрузка набора данных Wine Quality из CSV-файла в DataFrame
df = pd.read_csv("../Module2/data/auction.csv")
df.drop(columns=["verification.time"], inplace=True)

# Определение признаков (X) и целевой переменной (y)
X = df.drop(columns=["verification.result"])
y = df["verification.result"].astype(int)

# Нормализация признаков с помощью StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Разделение данных на обучающий и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

# Создание модели CatBoostClassifier
catboost = CatBoostClassifier(silent=True)

# Определение сетки параметров для перебора (настройка параметров CatBoost можно провести здесь)
param_grid = {
    "iterations": [100, 200, 300],  # пример настройки числа итераций
    "learning_rate": [0.01, 0.1, 0.2],  # пример настройки скорости обучения
}

# Использование GridSearchCV для поиска наилучших параметров модели
grid_search = GridSearchCV(catboost, param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Вывод наилучших параметров и оценки на кросс-валидации
print("Наилучшие параметры на кросс-валидации:", grid_search.best_params_)
print("Наилучшая оценка на кросс-валидации:", grid_search.best_score_)

# Прогнозирование на тестовом наборе с помощью модели с оптимальными параметрами
y_pred = grid_search.predict(X_test)

# Вычисление f1 на тестовом наборе
f1 = f1_score(y_pred, y_test)

# Вывод значения f1
print("f1 на тестовом наборе: ", f1)

Наилучшие параметры на кросс-валидации: {'iterations': 200, 'learning_rate': 0.1}
Наилучшая оценка на кросс-валидации: 0.995103281364327
f1 на тестовом наборе:  0.9908256880733944


# Набор данных "Predict students' dropout and academic success"

Датасет был создан в рамках проекта, направленного на снижение количества студентов, прекращающих обучение или испытывающих неудачи в высшем образовании, путем использования методов машинного обучения для выявления студентов, находящихся в рискованных ситуациях на раннем этапе их учебного пути, чтобы разработать стратегии для их поддержки.

Датасет включает информацию, известную на момент поступления студента в учебное заведение, такую как учебный путь, демографические данные и социоэкономические факторы.

Задача сформулирована как задача классификации с тремя категориями (бросившие обучение, зачисленные и выпускники) к концу нормативного срока обучения. То есть, для каждого студента предсказывается, к какой из трех категорий он относится к моменту окончания курса (нормативного срока обучения).

Краткое описание признаков можно посмотреть по ссылке: https://archive.ics.uci.edu/dataset/697/predict+students+dropout+and+academic+success

Целевая переменная:

- **Target**: бросившие обучение, зачисленные и выпускники

In [9]:
"""
По аналогии с решением задачи Auction Verification решите в текущей ячейке задачу для предсказания целевой переменной на 
наборе Bike Sharing Demand. С случае затруднений можно посмотреть на образец решения задания в ячейке ниже.
"""

# НАЧАЛО ВАШЕГО РЕШЕНИЯ
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score

numeric_features = []
categorical_features = []
df = pd.read_csv("../Module2/data/students_predictions.csv", sep=";")

X = df.drop(columns=["Target"])
y = df["Target"]

encoder = LabelEncoder()
y = encoder.fit_transform(y)
for col in X.columns:
    if X[col].nunique() < 50:
        categorical_features.append(col)
    else:
        numeric_features.append(col)

numeric_transformer = Pipeline(steps=[("num", StandardScaler())])
categorical_transformer = Pipeline(
    steps=[("cat", OneHotEncoder(drop="first", handle_unknown="ignore"))]
)

preprocessing = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=42, test_size=0.2
)

catboost = CatBoostClassifier(silent=True)

param_grid = {
    "iterations": [100, 200, 300],  # пример настройки числа итераций
    "learning_rate": [0.01, 0.1, 0.2],  # пример настройки скорости обучения
}
grid_search = GridSearchCV(catboost, param_grid, cv=5)
grid_search.fit(preprocessing.fit_transform(X_train), y_train)

y_pred = grid_search.predict(preprocessing.transform(X_test))

f1 = f1_score(y_test, y_pred, average="weighted")
# Вывод наилучших параметров и оценки на кросс-валидации
print("Наилучшие параметры на кросс-валидации:", grid_search.best_params_)
print("Наилучшая оценка на кросс-валидации:", grid_search.best_score_)

# Прогнозирование на тестовом наборе с помощью модели с оптимальными параметрами
y_pred = grid_search.predict(preprocessing.transform(X_test))

# Вычисление f1 на тестовом наборе
f1 = f1_score(y_pred, y_test, average="weighted")

# Вывод значения f1
print("f1 на тестовом наборе: ", f1)
# КОНЕЦ ВАШЕГО РЕШЕНИЯ

Наилучшие параметры на кросс-валидации: {'iterations': 300, 'learning_rate': 0.1}
Наилучшая оценка на кросс-валидации: 0.777906168340805
f1 на тестовом наборе:  0.7663372521423889


/home/fitlemon/code/math_for_ds/.venv/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:228: UserWarning: Found unknown categories in columns [6, 7, 8, 10, 17, 19, 24, 25, 26, 27] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/home/fitlemon/code/math_for_ds/.venv/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:228: UserWarning: Found unknown categories in columns [6, 7, 8, 10, 17, 19, 24, 25, 26, 27] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [4]:
hide_toggle(for_next=True)

In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score

# Загрузка набора данных bike sharing
numeric_features = []
categorical_features = []
df = pd.read_csv("../Module2/data/students_predictions.csv", sep=";")

# Отделяем целевую переменную от признаков
X = df.drop(columns=["Target"])
y = df["Target"]

for column in X.columns:
    unique_values_count = df[column].nunique()
    if 2 < unique_values_count < 50:
        categorical_features.append(column)
    else:
        numeric_features.append(column)

numeric_transformer = Pipeline(steps=[("scaler", StandardScaler())])

categorical_transformer = Pipeline(
    steps=[("onehot", OneHotEncoder(drop="first", handle_unknown="ignore"))]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

# Разделение данных на обучающий и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Создание модели CatBoostClassifier
catboost = CatBoostClassifier(silent=True)

# Определение сетки параметров для перебора (настройка параметров CatBoost можно провести здесь)
param_grid = {
    "iterations": [100, 200, 300],  # пример настройки числа итераций
    "learning_rate": [0.01, 0.1, 0.2],  # пример настройки скорости обучения
}

# Использование GridSearchCV для поиска наилучших параметров модели
grid_search = GridSearchCV(catboost, param_grid, cv=5)
grid_search.fit(preprocessor.fit_transform(X_train), y_train)

# Вывод наилучших параметров и оценки на кросс-валидации
print("Наилучшие параметры на кросс-валидации:", grid_search.best_params_)
print("Наилучшая оценка на кросс-валидации:", grid_search.best_score_)

# Прогнозирование на тестовом наборе с помощью модели с оптимальными параметрами
y_pred = grid_search.predict(preprocessor.transform(X_test))

# Вычисление f1 на тестовом наборе
f1 = f1_score(y_pred, y_test, average="weighted")

# Вывод значения f1
print("f1 на тестовом наборе: ", f1)

Наилучшие параметры на кросс-валидации: {'iterations': 200, 'learning_rate': 0.2}
Наилучшая оценка на кросс-валидации: 0.7711245095453856
f1 на тестовом наборе:  0.760958400402589


/home/fitlemon/code/math_for_ds/.venv/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:228: UserWarning: Found unknown categories in columns [5, 6, 7, 9, 10, 11, 16, 17, 18, 19] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


# Набор данных "accelerometer_gyro_mobile_phone_dataset"

Данные, собранные в 2022 году в Университете короля Сауда в Эр-Рияде для распознавания действий человека с помощью датчиков IMU мобильного телефона (акселерометр и гироскоп). эти действия сводятся к стоянию (остановке) или ходьбе.

Краткое описание признаков:
- accX	Вещественный
- accY	Вещественный
- accZ	Вещественный
- gyroX	Вещественный
- gyroY	Вещественный
- gyroZ	Вещественный
- timestamp	Datetime

Целевая переменная:

- **Activity**: стояние или ходьба

In [13]:
"""
По аналогии с решением задачи Auction Verification решите в текущей ячейке задачу для предсказания целевой переменной на 
наборе accelerometer_gyro_mobile_phone_dataset. С случае затруднений можно посмотреть на образец решения задания в ячейке ниже.
"""

# НАЧАЛО ВАШЕГО РЕШЕНИЯ
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score

df = pd.read_csv("../Module2/data/accelerometer_gyro_mobile_phone_dataset.csv")

X = df.drop(columns=["timestamp", "Activity"])
y = df["Activity"]

scaler = StandardScaler()

X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=42, test_size=0.2
)

catboost = CatBoostClassifier(silent=True)

param_grid = {
    "iterations": [100, 200, 300],  # пример настройки числа итераций
    "learning_rate": [0.01, 0.1, 0.2],  # пример настройки скорости обучения
}

grid_search = GridSearchCV(catboost, param_grid, cv=5)

grid_search.fit(scaler.fit_transform(X_train), y_train)

y_pred = grid_search.predict(scaler.transform(X_test))

f1 = f1_score(y_test, y_pred)

# Вывод наилучших параметров и оценки на кросс-валидации
print("Наилучшие параметры на кросс-валидации:", grid_search.best_params_)
print("Наилучшая оценка на кросс-валидации:", grid_search.best_score_)
# Вывод значения f1
print("f1 на тестовом наборе: ", f1)
# КОНЕЦ ВАШЕГО РЕШЕНИЯ

Наилучшие параметры на кросс-валидации: {'iterations': 200, 'learning_rate': 0.2}
Наилучшая оценка на кросс-валидации: 0.983315115109934
f1 на тестовом наборе:  0.9929638706617123


In [8]:
hide_toggle(for_next=True)

In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score

# Загрузка набора данных Appliances Energy Prediction
df = pd.read_csv("../Module2/data/accelerometer_gyro_mobile_phone_dataset.csv")

# Удаление исходного столбца "timestamp"
df = df.drop(columns=["timestamp"])

# Отделяем целевую переменную от признаков
X = df.drop(columns=["Activity"])
y = df["Activity"]

# Предобработка данных: преобразование категориальных признаков и нормализация числовых признаков
numeric_features = X.columns.tolist()

numeric_transformer = Pipeline(steps=[("scaler", StandardScaler())])

preprocessor = ColumnTransformer(
    transformers=[("num", numeric_transformer, numeric_features)]
)

# Разделение данных на обучающий и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Создание модели CatBoostClassifier
catboost = CatBoostClassifier(silent=True)

# Определение сетки параметров для перебора (настройка параметров CatBoost можно провести здесь)
param_grid = {
    "iterations": [100, 200, 300],  # пример настройки числа итераций
    "learning_rate": [0.01, 0.1, 0.2],  # пример настройки скорости обучения
}

# Использование GridSearchCV для поиска наилучших параметров модели
grid_search = GridSearchCV(catboost, param_grid, cv=5)
grid_search.fit(preprocessor.fit_transform(X_train), y_train)

# Вывод наилучших параметров и оценки на кросс-валидации
print("Наилучшие параметры на кросс-валидации:", grid_search.best_params_)
print("Наилучшая оценка на кросс-валидации:", grid_search.best_score_)

# Прогнозирование на тестовом наборе с помощью модели с оптимальными параметрами
y_pred = grid_search.predict(preprocessor.transform(X_test))

# Вычисление f1 на тестовом наборе
f1 = f1_score(y_pred, y_test)

# Вывод значения f1
print("f1 на тестовом наборе: ", f1)

Наилучшие параметры на кросс-валидации: {'iterations': 200, 'learning_rate': 0.2}
Наилучшая оценка на кросс-валидации: 0.983315115109934
f1 на тестовом наборе:  0.9929638706617123
